In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-02-14 06:57:33--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-02-14 06:57:34--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucab82c64bcce3940bd708e00411.dl.dropboxusercontent.com/cd/0/inline/BI4cpeNn5B65M_thG85TRAVDZ55NpxxOYvcaz0Q_iYwzGwOcfCp4c6ZTg-ns9HEsZL6uhzYklVHMzzzngMSmASioTvOBEOGv38Kb4l-tST8IP2K3DisabZP_Pz5iL8g14rI/file# [following]
--2021-02-14 06:57:34--  https://ucab82c64bcce3940bd708e00411.dl.dropboxusercontent.com/cd/0/inline/BI4cpeNn5B65M_thG85TRAVDZ55NpxxOYvcaz0Q_iYwzGwOcfCp4c6ZTg-ns9HEsZL6uhzYk

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout

In [4]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )

In [5]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"First off, I had my doubts just looking at the DVD box and reading it saying that it was about of bunch of teens gathering at a lake where they will find do or something. Any movie that has a premise like this has failed miserably, even as a slasher movie, except for the first Friday the 13th.  I wanted to get up and stop watching the movie at least 10 times, but I just kept thinking that it had to get a little better. It didn't. Usually, I think every movie has something that you can take from it. This has nothing.  Do yourself a favor, and find something constructive to do for 80 minutes. Like, give yourself papercuts, or eat dirt."
0


In [6]:
model = Sequential()

In [7]:
model.add(Input(shape=(1,), dtype="string"))

In [8]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_len,
)

In [9]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [10]:
model.add(Embedding(max_tokens + 1, 128))

model.add(GRU(64))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [11]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 24s 20ms/step - loss: 0.6514 - accuracy: 0.5818
Epoch 2/10
782/782 [==============================] - 16s 20ms/step - loss: 0.4430 - accuracy: 0.7983
Epoch 3/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3951 - accuracy: 0.8209
Epoch 4/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3690 - accuracy: 0.8325
Epoch 5/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3455 - accuracy: 0.8437
Epoch 6/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3248 - accuracy: 0.8532
Epoch 7/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3061 - accuracy: 0.8597
Epoch 8/10
782/782 [==============================] - 16s 20ms/step - loss: 0.2825 - accuracy: 0.8699
Epoch 9/10
782/782 [==============================] - 16s 20ms/step - loss: 0.2676 - accuracy: 0.8766
Epoch 10/10
782/782 [==============================] - 16s 20ms/step - loss: 0.251

In [13]:
model.evaluate(test_data)

782/782 [==============================] - 12s 14ms/step - loss: 0.5816 - accuracy: 0.7692


[0.5815908908843994, 0.7692000269889832]

In [14]:
text = "I loved the movie !"

In [15]:
model.predict([text])

array([[0.96465665]], dtype=float32)